In [2]:
import numpy as np
from secante import metodo_secante

In [9]:
import numpy as np
from secante import metodo_secante

def boonetRec(n, x):
    if n <= 1:
        raise ValueError("El valor de n debe ser mayor que 1.")
    
    val0, val1 = 1, x
    
    for j in range(1, n):
        val_next = ((2*j+1)*x*val1 - j*val0)/(j+1.)
        val0, val1 = val1, val_next 
    return val0, val1

def legendre(n, x):
    if isinstance(x, (int, float)):
        x = np.array([x])  # convertir a array de NumPy si no lo es
    elif not isinstance(x, np.ndarray):
        raise TypeError("El argumento 'x' debe ser un escalar o un array de NumPy.")
    
    if n == 0:
        valn = np.ones_like(x)
        dvaln = np.zeros_like(x)
    elif n == 1:
        valn = x
        dvaln = np.ones_like(x)
    else:
        valn_m1, valn = boonetRec(n, x)
        dvaln = n*(valn_m1-x*valn)/(1.-x**2)  # derivada
    return valn, dvaln

def legroots(n, delt=.2, Nit=1000, error='dist', eps=1e-05):
    roots = np.zeros(n)
    npos = n // 2  # pq son simétricos
    
    f = lambda x: legendre(n, x)[0]  # recordar que da dos salidas y quiero solo el Pn
    for i in range(npos):
        p0 = np.cos(np.pi * (4 * i + 3) / (4 * n + 2))  # semilla
        p1 = p0 + delt
        root = metodo_secante(f, p0, p1, eps, Nit)
        roots[i] = -root[0]  # Aquí se corrige
        roots[n - 1 - i] = root[0]  # Corrección: asignación simétrica
    return roots

def metodo_secante(f, x0, x1, eps, Nit):
    for _ in range(Nit):
        x_next = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        if abs(x_next - x1) < eps:
            return x_next
        x0, x1 = x1, x_next
    raise ValueError("El método de la secante no convergió.")

def gau_param(n, delt=.2, Nit=1000, error='dist', eps=1e-05): 
    xroot = legroots(n, delt=delt, Nit=Nit, error=error, eps=eps)
    dPn = legendre(n, xroot)[1]
    cj = 2/(1-xroot**2)/dPn**2
    return xroot, cj

def gauInt(f, interv, Npts, delt=0.2, Nit=1000, error='dist', eps=1e-05):
    a, b = min(interv), max(interv)
    
    # cantidades necesarias
    xs, cs = gau_param(Npts, delt=delt, Nit=Nit, error=error, eps=eps)
    
    # términos que escalan a xk
    coeffp = (b + a) / 2
    coeffm = (b - a) / 2

    
    ts = coeffp + coeffm*xs  # introducimos la nueva variable t 
    fk = cs*f(ts)
    val = coeffm*np.sum(fk)
    return val


In [10]:
def f(x):
    return 1/np.sqrt(x**2+1)

valComp = 0.88137358702
print('Valor ', valComp)
interv = [0., 1.]
for Npts in range(2, 10):
    val = gauInt(f, interv, Npts, eps=1e-11)  # notar como eps me da una noción de las cifras significativas
    print('===> ', Npts, 'valor ', val, 'Abs', abs(valComp-val))

Valor  0.88137358702
===>  2 valor  0.8817898064445449 Abs 0.0004162194245448392
===>  3 valor  0.881331201937916 Abs 4.238508208398439e-05
===>  4 valor  0.8813752230725129 Abs 1.6360525129099912e-06
===>  5 valor  0.8813735706987256 Abs 1.6321274443065192e-08
===>  6 valor  0.8813735849145594 Abs 2.1054405996423498e-09
===>  7 valor  0.8813735871721439 Abs 1.5214385307160683e-10
===>  8 valor  0.881373587014753 Abs 5.247025036680952e-12
===>  9 valor  0.8813735870195227 Abs 4.772848782863548e-13
